In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from feature_summary import export_feature_summary
from round_to_nearest import round_to_nearest_multiple
from split_dataset_by_missing_and_type import *
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import hashlib
from sklearn.preprocessing import PowerTransformer
from clean_data_helper import *
from preprocessing_helper import *

In [2]:
data = pd.read_csv('final_proj_data.csv')
test_data = pd.read_csv('final_proj_test.csv')
print(data.shape)
print(test_data.shape)

(10000, 231)
(2500, 230)


In [3]:
cleaned_data = remove_columns_with_missing_values(data, 0.25)
cleaned_data = remove_rows_with_missing_values(cleaned_data)
cleaned_data.to_csv('cleaned_data.csv', index=False)

New dataset shape after column removal: (10000, 67)
New dataset shape after removing rows with more than 30% missing columns: (9080, 67)


In [4]:
numeric_df, categorical_df = split_dataset(cleaned_data)
# numeric_df.to_csv('numeric_df.csv', index=False)
# categorical_df.to_csv('categorical_df.csv', index=False)

### Imputation

#### Numericals

In [5]:
df_with_missing, df_without_missing = split_dataframe_by_missing_values(numeric_df)
export_feature_summary(df_with_missing, 'num_sum_missing.csv', unique_threshold = 15)
df_with_missing.to_csv('df_with_missing.csv', index=False)

Feature summary exported to num_sum_missing.csv


In [6]:
result = get_imputation_feature_lists_from_dataset(df_with_missing, missing_threshold=0.2)
print(result)

{'low_unique_features': ['Var7'], 'moderate_unique_features': ['Var65', 'Var144'], 'high_unique_features': ['Var6', 'Var13', 'Var21', 'Var24', 'Var74', 'Var81', 'Var109', 'Var119', 'Var125', 'Var140', 'Var149'], 'high_missing_features': []}


In [7]:
# Assuming df_with_missing is a subset of another DataFrame `df`
# Ensure you create a copy to avoid SettingWithCopyWarning
df_with_missing = df_with_missing.copy()

# Define feature groups
low_unique_features = ['Var7']
moderate_unique_features = ['Var65', 'Var144']
high_unique_features = ['Var6', 'Var13', 'Var21', 'Var24', 'Var74', 
                        'Var81', 'Var109', 'Var119', 'Var125', 
                        'Var140', 'Var149']

# Numerical variables with a common divisor of 7
numerical_divisor_7_group = ['Var7']

# Numerical variables with a common divisor of 9
numerical_divisor_9_group = ['Var144', 'Var65']

# Imputation for low unique features (Mode Imputation)
mode_imputer = SimpleImputer(strategy='most_frequent')
df_with_missing.loc[:, low_unique_features] = mode_imputer.fit_transform(
    df_with_missing[low_unique_features]
)

# Imputation for moderate unique features (Median Imputation)
median_imputer = SimpleImputer(strategy='median')
df_with_missing.loc[:, moderate_unique_features] = median_imputer.fit_transform(
    df_with_missing[moderate_unique_features]
)

# Imputation for high unique features (Mean Imputation)
mean_imputer = SimpleImputer(strategy='mean')
df_with_missing.loc[:, high_unique_features] = mean_imputer.fit_transform(
    df_with_missing[high_unique_features]
)

# Apply rounding to nearest multiple of 7 for Var7
df_with_missing.loc[:, numerical_divisor_7_group] = df_with_missing[
    numerical_divisor_7_group
].apply(lambda x: round_to_nearest_multiple(x, 7))

# Apply rounding to nearest multiple of 9 for Var144 and Var65
for var in numerical_divisor_9_group:
    df_with_missing.loc[:, var] = df_with_missing[var].apply(
        lambda x: round_to_nearest_multiple(x, 9)
    )
print(df_with_missing.head())

     Var6  Var7   Var13  Var21  Var24  Var65  Var74      Var81  Var109  \
0   812.0  14.0  1252.0  156.0    0.0   27.0   14.0  227693.10    32.0   
1  2688.0   7.0  8820.0  364.0    4.0    9.0  210.0   17662.35   112.0   
2  1015.0  14.0  1784.0  136.0    2.0   18.0   98.0  190181.10    32.0   
3   168.0   0.0     0.0   24.0    0.0    9.0    0.0  348843.00    16.0   
4    14.0   0.0     0.0   36.0    0.0    9.0    0.0  235971.00     8.0   

   Var119   Var125  Var140  Var144    Var149  
0   525.0   4743.0   410.0    27.0       0.0  
1  1065.0  44541.0    60.0     9.0  398034.0  
2   625.0  14751.0  5720.0    27.0  554421.0  
3   275.0      0.0     0.0     9.0       0.0  
4    45.0      0.0     0.0     0.0       0.0  


In [8]:
# Concatenate df_with_missing and df_without_missing by columns (axis=1)
imputed_numeric_df = pd.concat([df_with_missing, df_without_missing], axis=1)
# print(imputed_numeric_df.isnull().sum())

#### Categorical

In [9]:
df_with_missing_cat, df_without_missing_cat = split_dataframe_by_missing_values(categorical_df)
export_feature_summary(df_with_missing_cat, 'df_with_missing_cat.csv', unique_threshold = 15)
df_with_missing_cat.to_csv('df_with_missing.csv', index=False)

Feature summary exported to df_with_missing_cat.csv


In [10]:
df_with_missing_cat = df_with_missing_cat.copy()

mode_imputing_columns = ['Var208', 'Var218', 'Var205', 'Var203', 'Var197']
mode_imputer = SimpleImputer(strategy='most_frequent')
df_with_missing_cat[mode_imputing_columns] = mode_imputer.fit_transform(df_with_missing_cat[mode_imputing_columns])

columns_to_impute_with_unknown = ['Var223', 'Var219', 'Var206']
df_with_missing_cat[columns_to_impute_with_unknown] = df_with_missing_cat[columns_to_impute_with_unknown].fillna('Unknown')

columns_to_impute_frequency = ['Var197', 'Var192', 'Var217']
df_with_missing_cat[columns_to_impute_frequency] = mode_imputer.fit_transform(df_with_missing_cat[columns_to_impute_frequency])

In [11]:
imputed_categroical_df = pd.concat([df_without_missing_cat, df_with_missing_cat], axis=1)
full_imputed_data = pd.concat([imputed_numeric_df, imputed_categroical_df], axis=1)
full_imputed_data.shape

(9080, 67)